In [1]:
import pandas as pd
import numpy as np
from langchain_community.llms import Ollama
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import YamlOutputParser

In [2]:
contexts_ids = pd.read_csv('../context_group.csv')
contexts_ids

,group_id,context
0,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,1,Following the disbandment of Destiny's Child i...
2,1,"A self-described 'modern-day feminist', Beyonc..."
3,2,Beyoncé attended St. Mary's Elementary School ...
4,2,"At age eight, Beyoncé and childhood friend Kel..."
5,2,The group changed their name to Destiny's Chil...
6,3,LeToya Luckett and Roberson became unhappy wit...
7,3,The remaining band members recorded 'Independe...
8,3,"In July 2002, Beyoncé continued her acting car..."
9,4,Beyoncé's first solo recording was a feature o...


In [3]:
contexts = pd.read_csv('../context.csv')
questions = pd.read_json('../train-v2.0.json')['data'].values
questions = questions[0]
questions

{'title': 'Beyoncé',
 'paragraphs': [{'qas': [{'question': 'When did Beyonce start becoming popular?',
     'id': '56be85543aeaaa14008c9063',
     'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
     'is_impossible': False},
    {'question': 'What areas did Beyonce compete in when she was growing up?',
     'id': '56be85543aeaaa14008c9065',
     'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
     'is_impossible': False},
    {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
     'id': '56be85543aeaaa14008c9066',
     'answers': [{'text': '2003', 'answer_start': 526}],
     'is_impossible': False},
    {'question': 'In what city and state did Beyonce  grow up? ',
     'id': '56bf6b0f3aeaaa14008c9601',
     'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
     'is_impossible': False},
    {'question': 'In which decade did Beyonce become famous?',
     'id': '56bf6b0f3aeaaa14008c9602',
     'answers': [{'text

In [4]:
import pickle 

save_path = './Automated_KG_list_Group.pkl'

def save_list_to_file(my_list, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(my_list, file)
    print(f'List saved to {file_path}')
    
def read_list_from_file(file_path):
    with open(file_path, 'rb') as file:
        my_list = pickle.load(file)
    print(f'List read from {file_path}')
    return my_list


In [5]:
llm = Ollama(model="gemma:7b")

question_mapping = {
    "1":[1,2,3],
    "2":[4,5,6],
    "3":[7,8,9],
    "4":[10,11,12],
    "5":[13,14],
    "6":[15,16,17],
    "7":[18,19,20],
    "8":[21,22],
    "9":[23,],
    "10":[24],
}

In [6]:
contexts = pd.read_csv('../context.csv')
questions = pd.read_json('../train-v2.0.json')['data'].values
questions = questions[0]
questions = questions['paragraphs'][:24]

In [7]:
questions

[{'qas': [{'question': 'When did Beyonce start becoming popular?',
    'id': '56be85543aeaaa14008c9063',
    'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
    'is_impossible': False},
   {'question': 'What areas did Beyonce compete in when she was growing up?',
    'id': '56be85543aeaaa14008c9065',
    'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
    'is_impossible': False},
   {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
    'id': '56be85543aeaaa14008c9066',
    'answers': [{'text': '2003', 'answer_start': 526}],
    'is_impossible': False},
   {'question': 'In what city and state did Beyonce  grow up? ',
    'id': '56bf6b0f3aeaaa14008c9601',
    'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
    'is_impossible': False},
   {'question': 'In which decade did Beyonce become famous?',
    'id': '56bf6b0f3aeaaa14008c9602',
    'answers': [{'text': 'late 1990s', 'answer_start': 276}],
    'is_imposs

In [8]:
from evaluate import load
exact_match_metric = load("exact_match")
automated_kg_list = read_list_from_file(save_path)

/Users/mertakcay/.pyenv/versions/3.9.18/envs/ollama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


List read from ./Automated_KG_list_Group.pkl


In [14]:
def extract_cypher_scripts(text):
    cypher_list = list()
    for line in text.split("\n"):
        if 'WHERE' in line or 'LIMIT' in line or 'MATCH' in line or 'RETURN' in line or 'CREATE' in line:
            cypher_list.append(line)
            
    return cypher_list

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", """Create a Cypher query to answer question in a Neo4j Knowledge Graph Without Explanation."""),
    ("user", "{Context}"),
    ("user", "{Question}"),
    ("user", "{KG}"),
    ])

for group_id, context_question_ids in question_mapping.items():
    group_id = int(group_id) - 1
    with open(f'./Queries/Group_{group_id}_Automated_LLM_Scripts.txt', 'w') as f:
        LLM_cypher_script = automated_kg_list[group_id]
        LLM_contexts = '\n'.join(contexts_ids[contexts_ids['group_id'] == group_id]['context'].to_list())
        for context_id in context_question_ids:
            for qas_question in questions[context_id-1]['qas']:
                sub_sub_sample_question = qas_question['question']
                chain = prompt | llm 
                output = chain.invoke({"Context": LLM_contexts, "Question":sub_sub_sample_question, "KG":'\n'.join(LLM_cypher_script)})
                output = extract_cypher_scripts(output)
                print(LLM_contexts, '\n', LLM_cypher_script, '\n', sub_sub_sample_question)
                print(output)
                f.write(sub_sub_sample_question)
                f.write('\n')
                f.write('\n'.join(output))
                f.write('\n')
                f.write('\n')
        print("---------")
    


 
 ["CREATE (b:Person {name: 'Beyoncé Giselle Knowles-Carter'})", "CREATE (a:Location {name: 'Houston, Texas'})", "CREATE (g:Group {name: 'Destiny's Child'})", "CREATE (m:Manager {name: 'Mathew Knowles'})", "CREATE (aw:Award {name: 'Grammy Award'})", "CREATE (alb:Album {name: 'Dangerously in Love'})", "CREATE (song:Song {name: 'Déjà Vu'})", "CREATE (movie:Movie {name: 'Dreamgirls'})", 'CREATE (b)<-[:HAS_MANAGER]-(m)', 'CREATE (b)<-[:LEADS_SINGER]-(g)', 'CREATE (song)<-[:CONTAINS_SONG]-(alb)', 'CREATE (movie)<-[:STARS_IN]-(b)'] 
 When did Beyonce start becoming popular?
["MATCH (b:Person {name: 'Beyoncé Giselle Knowles-Carter'})", 'RETURN b.createdAt']
 
 ["CREATE (b:Person {name: 'Beyoncé Giselle Knowles-Carter'})", "CREATE (a:Location {name: 'Houston, Texas'})", "CREATE (g:Group {name: 'Destiny's Child'})", "CREATE (m:Manager {name: 'Mathew Knowles'})", "CREATE (aw:Award {name: 'Grammy Award'})", "CREATE (alb:Album {name: 'Dangerously in Love'})", "CREATE (song:Song {name: 'Déjà Vu'})